In [40]:
import numpy as np
from sklearn import preprocessing
import pandas

from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.cross_validation import KFold
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn import metrics


from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, precision_score

In [20]:
#imports data and deletes unknown

CA_data_all = pandas.read_csv('casualty_train.csv')

CA_data = CA_data_all.loc[CA_data_all.age != 'Unknown']
CA_data.loc[np.logical_not(CA_data.pedestrian_location == CA_data.pedestrian_location), 'pedestrian_location'] = ''
CA_data.loc[np.logical_not(CA_data.pedestrian_movement == CA_data.pedestrian_movement), 'pedestrian_movement'] = ''
CA_data.severe = CA_data.severe.astype('int')


encode_data = DataFrameMapper([
    ('casualty_class', preprocessing.LabelBinarizer()),
    ('gender', preprocessing.LabelBinarizer()),
    (['age'], preprocessing.StandardScaler()),
    ('travel', preprocessing.LabelBinarizer()),
#     ('pedestrian_location', preprocessing.LabelBinarizer()),
#      ('pedestrian_movement', preprocessing.LabelBinarizer()),   
    (['year'], preprocessing.StandardScaler()),
        
], df_out= True)

X_headers = ['casualty_class','gender','age','pedestrian_location','pedestrian_movement','travel','year']
Y_CA = np.array(CA_data.loc[:,'severe'], dtype=int)

X_CA_pre_encode = CA_data.loc[:,X_headers]
X_CA = encode_data.fit_transform(X_CA_pre_encode)
X_CA.head()

,casualty_class_driver,casualty_class_passenger,casualty_class_pedestrian,gender,age,travel_bicycle,travel_bus,travel_car,travel_motorbike,travel_other,travel_pedestrian,travel_taxi,year
0,0.0,1.0,0.0,0.0,-0.136637,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.86088
1,0.0,1.0,0.0,0.0,-0.920083,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.54706
2,0.0,1.0,0.0,1.0,1.008400,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.20397
3,0.0,1.0,0.0,0.0,-1.100879,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.85457
4,0.0,1.0,0.0,0.0,-0.920083,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.16839


In [21]:
(x_train, x_test, y_train, y_test) = train_test_split(X_CA, Y_CA, test_size=0.1, random_state=5, stratify=CA_data.casualty_class)


In [39]:

svc = LinearSVC(class_weight='balanced', verbose=True)
logistic = linear_model.LogisticRegression() 

pipeline1 = Pipeline([('select', SelectKBest(f_classif)),
                       ("svc", svc)])

scores = []

for K in range(1,14):
    
#     print pipeline1.get_params()
    pipeline1.set_params(select__k=K)
    pipeline1.fit(x_train, y_train)
    

    # Evaluate the models using crossvalidation
    scores.append(np.mean(cross_val_score(pipeline1, x_test, y_test, scoring="f1", cv=3)))
    
print np.argmax(scores)
print np.max(scores)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]11
0.267846961644


In [42]:
y_pred = pipeline1.predict(x_test)

print classification_report(y_test, y_pred)
print
print accuracy_score(y_test, y_pred)
print
print roc_auc_score(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.94      0.60      0.73     19336
          1       0.17      0.67      0.27      2325

avg / total       0.85      0.61      0.68     21661


0.607682009141

0.633438924998
